# Deploying your model

The main goal of training a model is to be able to perform predictions on real world data, with the expectation that it will return similiar results to the training data.

In small orgs, serialization and deserialization suffices for deploying a model (info on that [here](https://flynn.gg/blog/machine-learning-model-serialization)), however large orgs need to convert a model into a format that can be used in most production environments (an api, website, offline + online applications).

Retraining a model everytime you need to use it is insane, so it's important to be able to save and load models. This is possible if the parameters (weights and biases) for each layer are saved into the `state_dict` dictionary

Here is some pseudocode of what saving a model would look like
```python
checkpoint = {"input": x_train.shape[1], "state_dict": model.state_dict()}
```

Once you've saved a checkpoint, you can use pytorchs `save` function:

```python
torch.save(checkpoint, "checkpoint.pth")
```

You can create a python file that imports pytorch libraries and contains the class that creates the network architecture of your model. This way you can conveniently load the model into a new worksheet, seperating the instantiation from the training.

To load the model, let's create a function that will perform three main actions.

1. load a checkpoint
2. load a model in a class
3. load the `state_dict` of that model

In [2]:
# here final_model is a local python file and .Classifier is a class stored in that file
def load_model_checkpoint(path):
    checkpoint = torch.load(path)
    model = final_model.Classifier(
        checkpoint["input"],
        checkpoint["output"],
        checkpoint["hidden"])
    model.load_state_dict(checkpoint["state_dict"])
    return model
model = load_model_checkpoint("checkpoint.pth")

NameError: name 'torch' is not defined

We can convert an existing pytorch model into a more efficient, production ready C++ model (Torchscript) with no python dependencies, using pytorchs `Just-In-Time` compiler module. You do this by passing your model, along with an example input, through the `torch.jit.trace` function.

```python
traced_script = torch.jit.trace(model, example)
```

Where model should contain the previously trained model, and `example` should contain the set of features you wish to feed your model in order to perform a prediction. This will return a script module, which can be used as a regular pytorch module.
```python
prediction = traced_script(input)
```